In [1]:
import pandas as pd
import numpy as np
import sys
import os
home = os.getcwd()[:os.getcwd().find('EnsembleCpG')]+'EnsembleCpG/'
os.environ["PYTHONPATH"] = home+"code/"
sys.path[1] = os.environ["PYTHONPATH"]
from common import commons
home = commons.home
from features_preprocess import get_winid
from common.commons import rename_features
import os
import re

INFO    ; 2019-01-01 08:03:48,825; root           ; commons:<module>;39: Existing logger at commons


In [2]:
def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,subset=['id'],inplace=True)
    return merged


In [3]:
dataset = commons.dataset
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type
with pd.HDFStore(home+'data/'+dataset+'/all_sites_winid','r') as h5s:
    all_sites = h5s['all_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [4]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [5]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

303
31
267
317
735
73
80


In [6]:
rename_features(all_sites)

In [7]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [9]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS','GWAVA']
#merge with additional features
with pd.HDFStore(feature_dir+'addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [10]:
#nearest tss distance    
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/xiaobo/git/EnsembleCpG/code/features_preprocess/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].apply(lambda x: int(x[3:]))
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/i

In [11]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,A549-BHLHE40,A549-CEBPB,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg12445832,1,834295.0,0.393497,0.530672,0.708097,0.0,4172.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.46,0.46,0.390,13.0,18658.0
1,cg12549180,1,847816.0,0.114105,0.735620,0.751967,0.0,4240.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.33,0.51,0.720,37.0,5137.0
2,cg00145911,1,867938.0,0.128342,0.720266,0.860667,0.0,4340.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.21,0.48,0.360,57.0,7408.0
3,cg19724344,1,877647.0,0.011710,0.913858,0.107180,0.0,4389.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.27,0.45,0.960,39.0,1936.0
4,cg22381068,1,906363.0,0.052980,0.818025,0.241956,0.0,4532.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.33,0.36,0.800,1.0,4216.0
5,cg01729262,1,933306.0,0.013356,0.908027,0.716371,0.0,4667.0,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.29,0.53,0.730,9.0,1036.0
6,cg06538872,1,947343.0,0.619045,0.431670,0.721224,0.0,4737.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.18,0.180,50.0,1504.0
7,cg20062691,1,949392.0,0.101880,0.749681,0.735959,0.0,4747.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.19,0.39,0.830,1.0,545.0
8,cg01711077,1,981222.0,0.104068,0.747098,0.611918,0.0,4907.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.32,0.22,0.600,0.0,25719.0
9,cg12612065,1,1023319.0,0.674250,0.411855,0.705664,0.0,5117.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.32,0.20,0.310,9.0,6121.0


In [12]:
with pd.HDFStore(home+'data/'+dataset+'/all_features','w') as h5s:
    h5s['all_features'] = all_sites

In [9]:
###all 450K sites features, only need to RUN ONCE
dataset = 'Cd'
feature_dir = home+'data/features/'+dataset+'/'
all_450_features = home+'data/'+dataset+'/all_450k_features'
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type


In [10]:
with pd.HDFStore(home+'data/'+dataset+'/all_450k_sites_winid','r') as h5s:
    all_sites = h5s['all_450k_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [11]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,start,winid,end
0,cg13869341,1,15865,0.0426,0.297000,0.862405,15801,80,16000
1,cg24669183,1,534242,-0.0561,0.421300,0.803156,534201,2672,534400
2,cg15560884,1,710097,-0.0058,0.788800,0.662852,710001,3551,710200
3,cg01014490,1,714177,-0.0243,0.375500,0.016537,714001,3571,714200
4,cg17505339,1,720865,-0.0853,0.027480,0.967550,720801,3605,721000
5,cg05898754,1,805102,0.0182,0.659200,0.177819,805001,4026,805200
6,cg03128332,1,805338,0.0092,0.882000,0.045307,805201,4027,805400
7,cg16619049,1,805541,0.0007,0.991300,0.283757,805401,4028,805600
8,cg05475702,1,812248,0.0401,0.038670,0.472167,812201,4062,812400
9,cg18147296,1,812539,-0.0007,0.976800,0.311583,812401,4063,812600


In [12]:
#feature_dir = home+'data/features/'+dataset.split('/')[0]+'/'
files = os.listdir(feature_dir)
pattern = '.*all_450k.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [13]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

317
31
735
303
73
267
80


In [14]:
rename_features(all_sites)

In [15]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [16]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS','GWAVA']
#merge with additional features
with pd.HDFStore(feature_dir+'all_450k_addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [18]:
#nearest tss distance 
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [20]:
with pd.HDFStore(all_450_features,'w') as h5s:
    h5s['all_450k_features'] = all_sites.drop(['pvalue','beta'],axis=1)

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['id', 'dist_to_nearest_tss']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
import pandas as pd
with pd.HDFStore('/home/ec2-user/volume/git/EnsembleCpG/data/RNASeq/ENCFF939IHN','r') as h5s:
    x = h5s['ENCFF939IHN']

In [14]:
x

,winid,ENCFF939IHN_RNASeq_counts
